In [40]:
def extract_cascades_info_subsample(vi,tiledir,tiles,nights,petals,subset_id):
  dataname = tiledir+'/'+tiles[0] + '/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-subset-'+str(subset_id)+'.fits'
  #tf = Table.read(dataname,hdu='FIBERMAP')
  tspec = Table.read(dataname,hdu='ZBEST')
  for i in range(1,len(petals)):
      #tfn = Table.read(tiledir+'/'+tiles[0] + '/deep/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-deep.fits',hdu='FIBERMAP')
      #tf = vstack([tf,tfn])
      tspecn = Table.read(tiledir+'/'+tiles[0] + '/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-subset-'+str(subset_id)+'.fits',hdu='ZBEST')
      tspec = vstack([tspec,tspecn])
            
  #EXPID = list(set(tf['EXPID']))[-1] 
  #'''This was 0 before but  '''
  #tf = tf[tf['EXPID']==EXPID]
  #tf_df = tf['TARGETID','TARGET_RA','TARGET_DEC','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','EBV'].to_pandas()
  #tf_df = tf.to_pandas()
  tspec_df = tspec['TARGETID','DELTACHI2','Z','ZWARN','ZERR','CHI2','NPIXELS','SPECTYPE'].to_pandas()
  vi = vi.merge(tspec_df, how='left', on='TARGETID',suffixes=('', '_cas'))
  #print(len(vi.columns))
  return vi

In [41]:
def return_truth_table_name(type_name, tileid):
    truth_table_filename = {'ELG':{'80608':'desi-vi_ELG_tile80608_nightdeep_merged_all_210208.csv','80606':'desi-vi_ELG_tile80606_nightdeep_merged_all_210218.csv'},
                           'LRG':{'80605':'desi-vi_LRG_tile80605_nightdeep_merged_all_210224.csv','80609':'desi-vi_LRG_tile80609_nightdeep_merged_all_21030.csv'},
                           'BGS':{'80613':'desi-vi_BGS_tile80613_nightdeep_merged_all_210202.csv'},
                           'QSO':{'80605':'desi-vi_QSO_tile80605_nightdeep_merged_all_210223_ADDING_object_info.csv','80607':'desi-vi_QSO_tile80607_nightdeep_merged_all_210214_ADDING_object_info.csv',
                                 '80609':'desi-vi_QSO_tile80609_nightdeep_merged_all_210210_ADDING_object_info.csv'}}
    VI_dir ='/global/cfs/cdirs/desi/sv/vi/TruthTables/Blanc/'+type_name+'/'
    return VI_dir+truth_table_filename[type_name][tileid]

In [42]:
def return_blanc_and_cascades_info(type_name,tileid,tiledir,subset_id):
    
    #tiledir   = '/global/cfs/cdirs/desi/spectro/redux/cascades/tiles/'
    file_name = return_truth_table_name(type_name,tileid)
    vi_old = pd.read_csv(file_name, delimiter = ",", engine='python', keep_default_na=False)
    #print(len(vi_old))
    nights = ['20201223']
    petals = ['0','1','2','3','4', '5', '6' ,'7', '8', '9']
    
    vi = extract_cascades_info_subsample(vi_old,tiledir,[tileid],nights,petals,subset_id)  
    
    vi['delta_delta']=vi['DELTACHI2']-vi['DELTACHI2_cas']
    vi['dz']=abs(vi['Redrock_z']-vi['best_z'])/(1.+vi['best_z'])
    vi['dz_cas']=abs(vi['Z']-vi['best_z'])/(1.+vi['best_z'])
    vi['subset_id']=subset_id
    return vi    

In [43]:
import pandas as pd
import os, sys, glob
import fnmatch
import matplotlib.pyplot as plt
import re
import numpy as np
from astropy.table import Table, join, vstack
import astropy.io.fits as pf
import pandas as pd
plt.rcParams['figure.figsize'] = [12, 8]

In [50]:
type_name_list = ['ELG','ELG','LRG','LRG','BGS','QSO','QSO','QSO']
tileid_list = ['80606','80608','80605','80609','80613','80605','80607','80609']
for i in range(0,len(type_name_list)):
    if (tileid_list[i]=='80606') | (tileid_list[i]=='80605'):
        dir_depth = '/global/cfs/cdirs/desi/users/rongpu/redux/cascades/3x_depth/'
    else:
        dir_depth = '/global/cfs/cdirs/desi/users/rongpu/redux/cascades/4x_depth/'
    data1 = return_blanc_and_cascades_info(type_name_list[i],tileid_list[i],dir_depth,'1')
    data2 = return_blanc_and_cascades_info(type_name_list[i],tileid_list[i],dir_depth,'2')

    total_data = data1.append(data2)
    total_data.to_csv(type_name_list[i]+'_'+tileid_list[i]+'_Cascades_subsamples.csv',index=False)
    
    #total_merge = data1.merge(data2, how='left', on='TARGETID',suffixes=('_1', '_2'))